### Autograd in PyTorch

Let's use the PyTorch automatic differentiation library to solve the regression problem that we used the Moore-Penrose Pseudoinverse to solve in the [*Machine Learning Foundations: Linear Algebra II: Matrix Operations* notebook](http://127.0.0.1:8888/notebooks/work/notebooks/2-linear-algebra-ii.ipynb).

In [ ]:
import torch

As in *Linear Algebra II*, in machine learning the convention is that each row $i$ represents an instance: 

In [ ]:
x = np.array([0, 1, 2, 3, 4, 5, 6, 7.]).reshape(-1, 1)
y = np.array([-.82, -.94, -.12, .26, .39, .64, 1.02, 1.]).reshape(-1, 1)

In [ ]:
x

In [ ]:
y

In [ ]:
fig, ax = plt.subplots()
_ = ax.scatter(x, y)

`nn.Linear` applies the transformation $y = xA^T + b$ where $A$ is matrix of weights and $b$ is vector of biases. In this example, there is only a single weight and a single bias ($y$-intercept) value because of how we've constrained the number of inputs and outputs to `linear` both to `1`:

In [ ]:
class LinearRegression(torch.nn.Module):
    
    # Superclassing of new Module is standard in PyTorch
    def __init__(self): 
        super(LinearRegression, self).__init__() 
        self.linear = torch.nn.Linear(1, 1)
    
    # Define the forward pass: 
    def forward(self, my_x):
        my_y = self.linear(my_x)
        return my_y

In [ ]:
model = LinearRegression()

This is the scalar in our regression model representing the slope $m$ between $y$ and $x$. In regression, it is often denoted as $\beta$ or $w$:

In [ ]:
m = model.linear.weight
m

This is the scalar in our regression model representing the $y$-intercept, which is typically denoted as $b$: 

In [ ]:
b = model.linear.bias
b

Prior to training, the model has a random slope $m$ and random intercept $b$: 

In [ ]:
def regression_plot(my_x, my_y, my_m, my_b):
    
    fig, ax = plt.subplots()

    ax.scatter(my_x, my_y)
    x_min, x_max = ax.get_xlim()
    y_min, y_max = my_b, my_b + my_m*(x_max-x_min)

    ax.plot([x_min, x_max], [y_min, y_max])
    _ = ax.set_xlim([x_min, x_max])

In [ ]:
regression_plot(x, y, m, b)

There is a PyTorch `MSELoss` method, but let's define it outselves to see how it works. MSE cost $C$ is defined by: $$C = \frac{1}{n} \sum_{i=1}^n (\hat{y_i}-y_i)^2 $$

In [ ]:
def mse_cost(my_yhat, my_y): 
    sigma = torch.sum((my_yhat - my_y)**2)
    return sigma/len(my_y)

In [ ]:
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

In [ ]:
x_pt = torch.from_numpy(x).float()

In [ ]:
x_pt

In [ ]:
x_pt.requires_grad_()

Forward pass:

In [ ]:
yhat = model(x_pt) 

In [ ]:
yhat

In [ ]:
y_pt = torch.from_numpy(y).float()

In [ ]:
cost = mse_cost(yhat, y_pt)
cost

Chain rule to differentiate w.r.t. model parameters:

In [ ]:
cost.backward() 

Use differentiation results to adjust model parameters in direction of lower cost: 

In [ ]:
optimizer.step()

In [ ]:
yhat = model(x_pt)
cost = mse_cost(yhat, y_pt)
cost

After a round of training, the model fits a tiny bit better, although it may be difficult to tell by eye: 

In [ ]:
regression_plot(x, y, model.linear.weight, model.linear.bias)

In [ ]:
epochs = 32
for epoch in range(epochs):
    
    optimizer.zero_grad() # Reset gradients to zero
    
    yhat = model(x_pt) 
    cost = mse_cost(yhat, y_pt) 
    
    cost.backward() 
    optimizer.step()
    
    print('Epoch {}, cost {}'.format(epoch, cost.item()))

In [ ]:
regression_plot(x, y, model.linear.weight, model.linear.bias)